Doc:
- [spark-packages.org](https://spark-packages.org/package/mongodb/mongo-spark)

In [1]:
%load_ext memory_profiler

In [2]:
from pyspark.sql.session import SparkSession

In [34]:
user = "root"
password = "secret"
host = "db.mongo.app.com"
database = "ds"
collection = "s4gt6dd"

uri = f"mongodb://{user}:{password}@{host}"

In [35]:
%%memit
spark = SparkSession.builder \
    .master("local") \
    .appName("SparkMongo") \
    .config("spark.mongodb.input.uri", uri) \
    .getOrCreate()

peak memory: 146.30 MiB, increment: 0.16 MiB


sc = spark.sparkContext
sc.setLogLevel("INFO")

In [69]:
from pymongo import MongoClient
import bson
from motor import MotorClient

In [71]:
async_client = MotorClient(uri)

In [37]:
client = MongoClient(uri)

In [38]:
client

MongoClient(host=['db.mongo.app.com:27017'], document_class=dict, tz_aware=False, connect=True)

`Cursor`: class to iterate over Mongo query results.

In [100]:
cursor = client.sc.s4gt6dd.find({})

In [103]:
client.sc.s4gt6dd.count_documents({})

3

In [101]:
cursor.batch_size(20)

In [113]:
cursor.rewind()

In [114]:
cursor.next()

{'_id': ObjectId('5d56fbe4ac21aa7abd9ca027')}

In [66]:
client.sc.s4gt6dd.insert_one(bson.SON())

In [74]:
fut = async_client.sc.s4gt6dd.insert_one(bson.SON())

In [76]:
fut.done()

True

In [65]:
elt = bson.SON()

In [77]:
elt["a"] = bson.int64.Int64(12)

In [78]:
elt

SON([('a', 12)])

In [81]:
elt?

Type:        SON
String form: SON([('a', 12)])
Length:      1
File:        /opt/conda/lib/python3.7/site-packages/bson/son.py
Docstring:  
SON data.

A subclass of dict that maintains ordering of keys and provides a
few extra niceties for dealing with SON. SON provides an API
similar to collections.OrderedDict from Python 2.7+.


In [64]:
elt = bson.BSON()

In [67]:
client.list_database_names()

['admin',
 'config',
 'local',
 'people-bson',
 'saildrone',
 'sc',
 'starwars',
 'test']

https://docs.mongodb.com/spark-connector/master/python/filters-and-sql/

# DB initialisation

In [119]:
from bson import SON, Int64, Binary, Decimal128
from datetime import datetime

In [151]:
elt = SON()

elt["time"] = datetime(2019, 1, 9, 13, 34, 23)
elt["elt1"] = {"name": "elt1", "unit": "unit1", "data": Int64(34)}
elt["elt2"] = 34
elt["elt3"] = float(4.3)
elt["elt4"] = 4.3
elt["elt5"] = Binary(b'azddqds')
elt["elt6"] = [1, 2, 3.4, 4, 5]

In [152]:
client.sc.s4gt6dd.insert_one(elt)

In [153]:
client.

SyntaxError: invalid syntax (<ipython-input-153-aa4d2e851f66>, line 1)

In [144]:
client.system.js.save( { _id : "Sum" ,
value : function(key,values)
{
    var total = 0;
    for(var i = 0; i < values.length; i++)
        total += values[i];
    return total;
}});

SyntaxError: invalid syntax (<ipython-input-144-22e873ec34d5>, line 3)

In [154]:
for elt in client.sc.s4gt6dd.find({}):
    print(elt)

{'_id': ObjectId('5d56fbe4ac21aa7abd9ca027')}
{'_id': ObjectId('5d56fcc3ac21aa7abd9ca029')}
{'_id': ObjectId('5d56fcd3ac21aa7abd9ca02a')}
{'_id': ObjectId('5d570d10ac21aa7abd9ca02b'), 'time': datetime.datetime(2019, 1, 9, 13, 34, 23)}
{'_id': ObjectId('5d570e11ac21aa7abd9ca02c'), 'time': datetime.datetime(2019, 1, 9, 13, 34, 23), 'elt1': 34, 'elt2': 34, 'elt3': 4.3, 'elt4': b'azddqds'}
{'_id': ObjectId('5d570e15ac21aa7abd9ca02d'), 'time': datetime.datetime(2019, 1, 9, 13, 34, 23), 'elt1': 34, 'elt2': 34, 'elt3': 4.3, 'elt4': b'azddqds'}
{'_id': ObjectId('5d5712c2ac21aa7abd9ca02e'), 'time': datetime.datetime(2019, 1, 9, 13, 34, 23), 'elt1': {'name': 'elt1', 'unit': 'unit1', 'data': 34}, 'elt2': 34, 'elt3': 4.3, 'elt4': 4.3, 'elt5': b'azddqds', 'elt6': [1, 2, 3.4, 4, 5]}
{'_id': ObjectId('5d5712d2ac21aa7abd9ca02f'), 'time': datetime.datetime(2019, 1, 9, 13, 34, 23), 'elt1': {'name': 'elt1', 'unit': 'unit1', 'data': 34}, 'elt2': 34, 'elt3': 4.3, 'elt4': 4.3, 'elt5': b'azddqds', 'elt6': [1

In [10]:
pipeline = [{
    "$project": {
        "_id": 0,
        "name": '$homeworld.name',
        "rotation_period": '$homeworld.rotation_period',
        "orbital_period": '$homeworld.orbital_period',
        "diameter": '$homeworld.diameter',
        "climate": '$homeworld.climate'
        }
    }
]

In [11]:
people = client.starwars.get_collection("people")

In [12]:
cursor = people.aggregate(pipeline)

In [13]:
cursor.next()

{'name': 'Tatooine',
 'rotation_period': 23,
 'orbital_period': 304,
 'diameter': 10465,
 'climate': 'arid'}

In [14]:
res = people.find_one()

In [15]:
cursor = people.find({})

In [33]:
cursor.collection

Collection(Database(MongoClient(host=['db.mongo.app.com:27017'], document_class=dict, tz_aware=False, connect=True), 'starwars'), 'people')

In [17]:
res.keys()

dict_keys(['_id', 'name', 'height', 'mass', 'hair_color', 'skin_color', 'eye_color', 'birth_year', 'gender', 'homeworld', 'films', 'species', 'vehicles', 'starships', 'created', 'edited', 'url'])

In [18]:
res['homeworld']

{'name': 'Tatooine',
 'rotation_period': 23,
 'orbital_period': 304,
 'diameter': 10465,
 'climate': 'arid',
 'gravity': '1 standard',
 'terrain': 'desert',
 'surface_water': 1,
 'population': 200000,
 'residents': ['https://swapi.co/api/people/1/',
  'https://swapi.co/api/people/2/',
  'https://swapi.co/api/people/4/',
  'https://swapi.co/api/people/6/',
  'https://swapi.co/api/people/7/',
  'https://swapi.co/api/people/8/',
  'https://swapi.co/api/people/9/',
  'https://swapi.co/api/people/11/',
  'https://swapi.co/api/people/43/',
  'https://swapi.co/api/people/62/'],
 'films': ['https://swapi.co/api/films/5/',
  'https://swapi.co/api/films/4/',
  'https://swapi.co/api/films/6/',
  'https://swapi.co/api/films/3/',
  'https://swapi.co/api/films/1/'],
 'created': '2014-12-09T13:50:49.641000Z',
 'edited': '2014-12-21T20:48:04.175778Z',
 'url': 'https://swapi.co/api/planets/1/'}

In [19]:
%%memit
df = spark.read.format("mongo").\
    option("spark.mongodb.input.database", database).\
    option("spark.mongodb.input.collection", collection).\
    option("pipeline", pipeline).\
    load()

peak memory: 65.05 MiB, increment: 0.09 MiB


In [20]:
df.show()

+---------+--------+--------+--------------+---------------+
|  climate|diameter|    name|orbital_period|rotation_period|
+---------+--------+--------+--------------+---------------+
|     arid|   10465|Tatooine|           304|             23|
|     arid|   10465|Tatooine|           304|             23|
|temperate|   12120|   Naboo|           312|             26|
|     arid|   10465|Tatooine|           304|             23|
|temperate|   12500|Alderaan|           364|             24|
|     arid|   10465|Tatooine|           304|             23|
|     arid|   10465|Tatooine|           304|             23|
|     arid|   10465|Tatooine|           304|             23|
|     arid|   10465|Tatooine|           304|             23|
|temperate|       0| Stewjon|       unknown|        unknown|
+---------+--------+--------+--------------+---------------+



In [21]:
df.distinct().show()

+---------+--------+--------+--------------+---------------+
|  climate|diameter|    name|orbital_period|rotation_period|
+---------+--------+--------+--------------+---------------+
|     arid|   10465|Tatooine|           304|             23|
|temperate|   12500|Alderaan|           364|             24|
|temperate|   12120|   Naboo|           312|             26|
|temperate|       0| Stewjon|       unknown|        unknown|
+---------+--------+--------+--------------+---------------+



In [22]:
df.printSchema()

root
 |-- climate: string (nullable = true)
 |-- diameter: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- orbital_period: string (nullable = true)
 |-- rotation_period: string (nullable = true)



In [23]:
df2 = df.distinct().toPandas()

In [24]:
df.createOrReplaceTempView("temp")

In [25]:
spark.sql("SELECT DISTINCT climate FROM temp WHERE diameter < 11000").show()

+---------+
|  climate|
+---------+
|temperate|
|     arid|
+---------+

